# Reonstruct paleoshapes
This notebook will calculate paleoshapes (shapes are defined by list of points) for a given list of modern shapes at a given list of ages.

## import pygplates and other packages

In [1]:
import os
import sys
# add pygplates to python part
sys.path.insert(0, os.path.abspath('./bin/pygplates_0.36.0_py37_Darwin-x86_64')) # macOS Intel 
import pygplates

import pandas as pd

## user input

In [2]:
# input csv files (one for each shape) with label, modern latitude, modern longitude
shapesNames = ['catskill', 'price-pocono', 'pennington']

# list of ages (in Ma) for which we want to reconstruct paleolocations for the input sites
ages           = ['385', '380', '375', '370', '366', '359', '354', '349', '344', '339', '333', '327']

## plate model
List of available models at http://portal.gplates.org/portal/rotation_models/.
Here we are using the 'PALEOMAP PaleoAtlas for GPlates'by Scotese et al. (https://www.earthbyte.org/paleomap-paleoatlas-for-gplates/)


In [3]:
# static polygons are the 'partitioning features'
static_polygons = pygplates.FeatureCollection('PALEOMAP_Global_Plate_Model/PALEOMAP_PlatePolygons.gpml')
# actual rotation model
rotation_model=pygplates.RotationModel('PALEOMAP_Global_Plate_Model/PALEOMAP_PlateModel.rot')

## reconstruct points
4-step process to reconstruct paleolocations:
1. combine input points into feature collection
2. create polygon from points
3. assign plate ids to features
4. reconstruct paleolocations for features

In [4]:
# reconstruct polygons, i.e. area shapes

for shapeCount, shapeName in enumerate(shapesNames):
    
    # load shapes coordinates
    df_shape = pd.read_csv('./'+ shapeName +'-shape.csv',sep=',')
    
    for ageCount, age in enumerate(ages):

        # Create target directory & all intermediate directories if don't exists
        dirName = './reconstructed-shapes/'+ ages[ageCount] + 'Ma'
        if not os.path.exists(dirName):
            os.makedirs(dirName)
            print("Directory " , dirName ,  " Created ")
        else:    
            print("Directory " , dirName ,  " already exists") 

        # put the points into a feature collection, using Lat,Lon coordinates from dataframe
        polygon_points = []
        polygon_feature = []
        for index,row in df_shape.iterrows():
            point = pygplates.PointOnSphere(float(row.LAT),float(row.LON))
            polygon_points.append(point)

        #add_polyline_feature_from_points(polyline_features, polyline_points, row)
        polygon = pygplates.PolygonOnSphere(polygon_points)
        polygon_feature = pygplates.Feature() # 'unclassified' feature
        polygon_feature.set_geometry(polygon)

        # The partition points function can then be used as before
        partitioned_polygon_feature = pygplates.partition_into_plates(static_polygons, rotation_model, polygon_feature) 

        # Reconstruct the shapes
        reconstructed_polyline_feature_geometries = []
        pygplates.reconstruct(partitioned_polygon_feature, rotation_model, dirName +'/' + shapeName + '-shape.shp', float(ages[ageCount]) ) 


Directory  ./reconstructed-shapes/385Ma  already exists
Directory  ./reconstructed-shapes/380Ma  already exists
Directory  ./reconstructed-shapes/375Ma  already exists
Directory  ./reconstructed-shapes/370Ma  already exists
Directory  ./reconstructed-shapes/366Ma  already exists
Directory  ./reconstructed-shapes/359Ma  already exists
Directory  ./reconstructed-shapes/354Ma  already exists
Directory  ./reconstructed-shapes/349Ma  already exists
Directory  ./reconstructed-shapes/344Ma  already exists
Directory  ./reconstructed-shapes/339Ma  already exists
Directory  ./reconstructed-shapes/333Ma  already exists
Directory  ./reconstructed-shapes/327Ma  already exists
Directory  ./reconstructed-shapes/385Ma  already exists
Directory  ./reconstructed-shapes/380Ma  already exists
Directory  ./reconstructed-shapes/375Ma  already exists
Directory  ./reconstructed-shapes/370Ma  already exists
Directory  ./reconstructed-shapes/366Ma  already exists
Directory  ./reconstructed-shapes/359Ma  already